In [1]:
import numpy as np
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn

In [5]:
df = pd.read_csv('NHIS_OPEN_GJ_2018_100.csv')

In [7]:
df_sample = df[['성별코드', '연령대코드(5세단위)','수축기혈압','이완기혈압','식전혈당(공복혈당)','신장(5Cm단위)', '체중(5Kg단위)', '허리둘레']]


In [8]:
df_sample = df_sample.dropna()

In [9]:
df_sample = df_sample.loc[(df_sample['허리둘레'] >= 40) & (df_sample['허리둘레'] <= 140)]

In [10]:
def bmi(height,weight):
    m = height / 100
    return weight / (m ** 2)
def diabetes(x):
    if x >= 100 and x < 126:
        return "전단계"
    elif x >= 127:
        return "당뇨"
    else:
        return "정상"

def fat(bmi) :
    if(bmi < 25):
        return '정상'
    elif(bmi >= 25 and bmi < 30):
        return '과체중'
    else:
        return '비만'

def pulse(low,high):
    if low >= 100 or high >= 160:
        return "2기고혈압"
    elif 90 <= low < 100 or 140 <= high < 160:
        return "1기"
    elif 80 <= low < 90 or 120 <= high < 140:
        return "전단계"
    else:
        return "정상"

def bokbu(gen, hurry):
    if gen == 1 and hurry >= 90:
        return "비만"
    elif gen == 2 and hurry >= 85:
        return "비만"
    else:
        return "정상"
        

In [11]:
df_sample['당뇨병'] = df_sample['식전혈당(공복혈당)'].map(diabetes)
df_sample['고혈압'] = np.vectorize(pulse)(df_sample['이완기혈압'], df_sample['수축기혈압'])
df_sample['bmi'] = np.vectorize(bmi)(df_sample['신장(5Cm단위)'], df_sample['체중(5Kg단위)'])
df_sample['비만도'] = df_sample['bmi'].map(fat)
df_sample['복부비만'] = np.vectorize(bokbu)(df_sample['성별코드'], df_sample['허리둘레'])

In [12]:
df_sample.describe()

,성별코드,연령대코드(5세단위),수축기혈압,이완기혈압,식전혈당(공복혈당),신장(5Cm단위),체중(5Kg단위),허리둘레,bmi
count,993643.000000,993643.000000,993643.000000,993643.000000,993643.000000,993643.000000,993643.000000,993643.000000,993643.000000
mean,1.463026,10.569117,122.673883,76.078374,100.874115,162.348681,63.633715,81.436899,24.010855
std,0.498631,2.861437,14.622733,10.010572,24.111816,9.312854,12.732016,9.677391,3.556839
min,1.000000,5.000000,63.000000,27.000000,23.000000,125.000000,30.000000,40.000000,13.333333
25%,1.000000,8.000000,112.000000,70.000000,89.000000,155.000000,55.000000,75.000000,21.484375
50%,1.000000,11.000000,121.000000,76.000000,96.000000,160.000000,60.000000,81.200000,23.875115
75%,2.000000,13.000000,131.000000,82.000000,105.000000,170.000000,70.000000,88.000000,26.122449
max,2.000000,18.000000,267.000000,173.000000,872.000000,190.000000,135.000000,140.000000,44.081633


In [13]:

def Entropy(df, attribute) :
    list1  = list(df[attribute].unique())
    etp = 0
    for i in range(len(list1)):
        q1 = len(df[df[attribute] == list1[i]])/len(df[attribute])
        etp -=  q1*np.log2(q1)
    return etp

def Group_info(df,m_attribute,fix_attribute):
    list2 = list(df[m_attribute].unique())
    info_Entropy = Entropy(df,fix_attribute)
    info_data = 0
    for i in range (len(list2)) :
        q2 = df.loc[df[m_attribute] == list2[i]]
        info_data +=  (len(q2[m_attribute])/len(df[m_attribute]))* Entropy(q2,fix_attribute) 
    
    data = info_Entropy - info_data
    
    return data

info = np.array([("성별코드"),("고혈압"),("비만도"),("복부비만"),("연령대코드(5세단위)")])

for i in info :
    print(i,":",Group_info(df_sample, i,"당뇨병") * 1000)

성별코드 : 14.315310579631646
고혈압 : 28.651320175026918
비만도 : 21.000867423723424
복부비만 : 20.98988639149235
연령대코드(5세단위) : 47.02802877504841


In [14]:
#지니 지수 https://lucy-the-marketer.kr/ko/growth/decision-tree-and-impurity/

In [15]:
def Gini(df, attribute) :
    list1  = list(df[attribute].unique())
    gini = 0
    for i in range(len(list1)):
        q1 = len(df[df[attribute] == list1[i]])/len(df[attribute])
        gini += (q1**2)
    return 1 - gini

def Group_Gini(df,m_attribute,fix_attribute):
    list2 = list(df[m_attribute].unique())
    info_data = 0
    for i in range (len(list2)) :
        q2 = df.loc[df[m_attribute] == list2[i]]
        info_data +=  (len(q2[m_attribute])/len(df[m_attribute]))* Gini(q2,fix_attribute) 
    
    return info_data

info = np.array([("성별코드"),("고혈압"),("비만도"),("복부비만"),("연령대코드(5세단위)")])

for i in info :
    print(i,":",Group_Gini(df_sample, i,"당뇨병"))

성별코드 : 0.5132477500810447
고혈압 : 0.5067788048087857
비만도 : 0.5102727536675177
복부비만 : 0.5110472000612207
연령대코드(5세단위) : 0.501566998920739
